In [4]:
class CMS:
    """
    Content Management System (CMS) client.
    
    This class integrates all CMS components into a unified interface for
    working with the conversation service API.
    """
    
    def __init__(
        self,
        app_id: str,
        env: Optional[str] = None,
        preferences: Optional[Dict[str, Any]] = None,
        env_mapping: Optional[Dict[str, str]] = None,
        log_level: str = "INFO",
        **kwargs,
    ) -> None:
        """
        Initialize the CMS client.
        
        Args:
            app_id: Application identifier string
            env: Environment name (dev, uat, prod, etc.)
            preferences: Dictionary of user preferences
            env_mapping: Dictionary mapping environment names to base URLs
            log_level: Loguru log level
            **kwargs: Additional arguments for configuration
        """
        # Configure logger
        logger.remove()
        logger.add(
            lambda msg: print(msg, end=""), 
            level=log_level, 
            format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>"
        )
        
        # Set up environment mapping if not provided
        if env_mapping is not None:
            self.env_mapping = env_mapping
        else:
            from src.comp.awm_genai.settings.base import env_mapping
            self.env_mapping = env_mapping
        
        # Basic properties
        self.app_id = app_id
        self.preferences = preferences if preferences is not None else {}
        
        # Set base URL based on environment
        self.base_url = self.env_mapping.get(env, self.env_mapping["default"])
        
        # Known error messages that may be returned by the API
        self.known_error_messages = [
            "Sorry, unable to process your query",
            "[keystone]",
        ]
        
        # Configure default timeouts and retry settings
        self.default_timeout = kwargs.get("default_timeout", 30)  # seconds
        self.long_timeout = kwargs.get("long_timeout", 5 * 60)  # 5 minutes
        self.max_retries = kwargs.get("max_retries", 5)
        self.retry_delay = kwargs.get("retry_delay", 5)  # seconds
        self.semaphore_limit = kwargs.get("semaphore_limit", 10)  # limit concurrent async operations
        
        # Initialize the request handler
        self.request_handler = RequestHandler(
            base_url=self.base_url,
            default_timeout=self.default_timeout,
            long_timeout=self.long_timeout,
            max_retries=self.max_retries,
            retry_delay=self.retry_delay,
            known_error_messages=self.known_error_messages,
            verify_ssl=False
        )
        
        # Initialize component modules
        self.preferences_manager = UserPreferencesManager(self.app_id, self.request_handler)
        self.conversation_manager = ConversationManager(
            self.app_id, 
            self.request_handler, 
            self.semaphore_limit
        )
        self.qa_service = QuestionAnswerService(
            self.app_id, 
            self.request_handler, 
            self.preferences_manager,
            self.preferences
        )
        
        logger.info(f"CMS client initialized for app {app_id} with environment {env or 'default'}")
        
    # ---- User Preferences Management ----

    def set_user_preferences(self, sso: str, preferences: Dict[str, Any]) -> bool:
        """
        Set user preferences for the current application.
        
        Args:
            sso: The sso authentication cookie value
            preferences: Dictionary of preferences to set
            
        Returns:
            True if successful, False otherwise
        """
        return self.preferences_manager.set_preferences(sso, preferences)
        
    async def async_set_user_preferences(self, sso: str, preferences: Dict[str, Any]) -> bool:
        """
        Asynchronously set user preferences for the current application.
        
        Args:
            sso: The sso authentication cookie value
            preferences: Dictionary of preferences to set
            
        Returns:
            True if successful, False otherwise
        """
        return await self.preferences_manager.async_set_preferences(sso, preferences)
        
    # ---- Conversation Management ----

    def get_all_conversations(self, sso: str) -> List[str]:
        """
        Retrieve all conversation IDs for the current user.
        
        Args:
            sso: The sso authentication cookie value
            
        Returns:
            List of conversation IDs
        """
        return self.conversation_manager.get_all_conversations(sso)

    async def async_get_all_conversations(self, sso: str) -> List[str]:
        """
        Asynchronously retrieve all conversation IDs for the current user.
        
        Args:
            sso: The sso authentication cookie value
            
        Returns:
            List of conversation IDs
        """
        return await self.conversation_manager.async_get_all_conversations(sso)

    def delete_conversation(self, sso: str, conversation_id: str) -> bool:
        """
        Delete a specific conversation.
        
        Args:
            sso: The sso authentication cookie value
            conversation_id: ID of the conversation to delete
            
        Returns:
            True if deletion was successful, False otherwise
        """
        return self.conversation_manager.delete_conversation(sso, conversation_id)

    async def async_delete_conversation(self, sso: str, conversation_id: str) -> bool:
        """
        Asynchronously delete a specific conversation.
        
        Args:
            sso: The sso authentication cookie value
            conversation_id: ID of the conversation to delete
            
        Returns:
            True if deletion was successful, False otherwise
        """
        return await self.conversation_manager.async_delete_conversation(sso, conversation_id)

    def delete_conversations(self, sso: str, ids: List[str]) -> List[str]:
        """
        Delete multiple conversations.
        
        Args:
            sso: The sso authentication cookie value
            ids: List of conversation IDs to delete
            
        Returns:
            List of successfully deleted conversation IDs
        """
        return self.conversation_manager.delete_conversations(sso, ids)

    async def async_delete_conversations(self, sso: str, ids: List[str]) -> List[str]:
        """
        Asynchronously delete multiple conversations.
        
        Args:
            sso: The sso authentication cookie value
            ids: List of conversation IDs to delete
            
        Returns:
            List of successfully deleted conversation IDs
        """
        return await self.conversation_manager.async_delete_conversations(sso, ids)

    def delete_all_conversations(self, sso: str) -> int:
        """
        Delete all conversations for the current user.
        
        Args:
            sso: The sso authentication cookie value
            
        Returns:
            Number of successfully deleted conversations
        """
        return self.conversation_manager.delete_all_conversations(sso)

    async def async_delete_all_conversations(self, sso: str) -> int:
        """
        Asynchronously delete all conversations for the current user.
        
        Args:
            sso: The sso authentication cookie value
            
        Returns:
            Number of successfully deleted conversations
        """
        return await self.conversation_manager.async_delete_all_conversations(sso)
        
    # ---- Question & Answer API ----
    
    def conversations_with_question(self, sso: str, question: str) -> Optional[str]:
        """
        Send a question to the conversation API and get a response.
        
        Args:
            sso: The sso authentication cookie value
            question: The question or prompt to send
            
        Returns:
            The answer from the API or None if the request failed
        """
        return self.qa_service.ask_question(sso, question)

    async def async_conversations_with_question(self, sso: str, question: str) -> Optional[str]:
        """
        Asynchronously send a question to the conversation API and get a response.
        
        Args:
            sso: The sso authentication cookie value
            question: The question or prompt to send
            
        Returns:
            The answer from the API or None if the request failed
        """
        return await self.qa_service.async_ask_question(sso, question)
            
    def inference(self, user_sso: str, question: str) -> Tuple[str, Dict[str, Any], Optional[str]]:
        """
        Make an inference request.
        
        Args:
            user_sso: The sso authentication cookie value
            question: The question or prompt to send
            
        Returns:
            Tuple containing (answer, metadata, error)
        """
        return self.qa_service.inference(user_sso, question)
            
    async def async_inference(self, user_sso: str, question: str) -> Tuple[str, Dict[str, Any], Optional[str]]:
        """
        Make an asynchronous inference request.
        
        Args:
            user_sso: The sso authentication cookie value
            question: The question or prompt to send
            
        Returns:
            Tuple containing (answer, metadata, error)
        """
        return await self.qa_service.async_inference(user_sso, question)
        
    # ---- Utility Methods ----
    
    def _create_headers(self, sso: str) -> Dict[str, str]:
        """
        Create standard headers for API requests.
        
        Args:
            sso: The sso authentication cookie value
            
        Returns:
            Dictionary of HTTP headers
        """
        return {"Cookie": f"sso={sso}", "app-id": self.app_id}

    def _get_payload(self, question: str) -> Dict[str, Any]:
        """
        Create the standard payload for conversation API requests.
        
        Args:
            question: The user's question or prompt
            
        Returns:
            Dictionary payload for API request
        """
        return {
            "conversation": {
                "title": f"{self.app_id}",
                "preferences": self.preferences,
            },
            "question": {"question": f"{question}"},
        }